# **Исследование данных из Яндекс Метрики**

In [4]:
# перед началом работы импортируем библиотеки, которые могут понадобиться в работе над проектом

import pandas as pd

import math

import numpy as np

from matplotlib import pyplot as plt

from scipy import stats as st

import seaborn as sns

import scipy.stats as stats

import plotly.express as px

import plotly.graph_objs as go

from plotly.subplots import make_subplots

import requests 

from urllib.parse import urlencode 

import re

from calendar import day_name

%matplotlib inline

In [5]:
# импорт библиотеки подключения к яндекс-метрике
from tapi_yandex_metrika import YandexMetrikaStats

In [6]:
from pandas.io.json import json_normalize
import json
from requests_html import HTMLSession
session = HTMLSession()

In [7]:
# токен доступа к метрике
ACCESS_TOKEN = "y0_AgAAAAADW_NUAAj2NwAAAADYMiX7i7Oo99btQp2L_O5Ao4wY7AUiVck"

# номер счетчика
count_number = "61617364"

api = YandexMetrikaStats(
    access_token=ACCESS_TOKEN)

In [8]:
# запишем в переменную требуемые метрики
current_metrics = "ym:s:visits,ym:s:bounceRate,ym:s:pageDepth,ym:s:avgVisitDurationSeconds, ym:s:sumGoalReachesAny "

## **Запрос с группировкой по источнику трафика**

In [10]:
# ЗАПРОС С ГРУППИРОВКОЙ ПО ИСТОЧНИКУ ТРАФИКА И ДЕТАЛЯМ ИСТОЧНИКА
# МЕТРИКИ: ВИЗИТЫ, % ОТКАЗОВ, СР ГЛУБИНА ПРОСМОТРА, СР СЕССИЯ, КОЛ-ВО ДОСТИЖЕНИЙ ЦЕЛИ

# параметры запроса
params = dict(
    ids=count_number,  # номер счетчика в ЯМ
    # метрики
    metrics=current_metrics,
    # группировки. Порядок группировки имеет значение!!
    dimensions="ym:s:date,ym:s:<attribution>TrafficSource,ym:s:<attribution>SourceEngine",
    attribution="lastSign",  # группировка по последнему значимому источнику
    date1='2022-06-20', 
    date2='2022-12-22',
    sort="ym:s:date",
    limit=10000,
    sccuracy = "full",      # максимальная точность отчетов
    proposed_accuracy = "false"     # система не может менять точность отчетов
)

#Получаем данные из Yandex.Metrika API
result = api.stats().get(params=params)
result = result().data

# из JSON файла оставляем только блок данных
result = result['data']

#Создаем пустой dict (словать данных)
dict_data = {}

#Парсим исходный list формата Json в dictionary (словарь данных)
for i in range(0, len(result)-1):
    dict_data[i] = {
            'date':result[i]["dimensions"][0]["name"],            # из списка dimensions, можно указывать не все параметры
            'traffic-source':result[i]["dimensions"][1]["name"],
            'traffic-details':result[i]["dimensions"][2]["name"],
            #'gender':result[i]["dimensions"][3]["name"],
            'visits':result[i]["metrics"][0],                     # из списа метрик
            'bounceRate':result[i]["metrics"][1],
            'pageDepth':result[i]["metrics"][2],
            'avgVisitSec':result[i]["metrics"][3],
            'Goals_total':result[i]["metrics"][4]
                   }

#Создаем DataFrame из dict (словаря данных или массива данных)
dict_keys = dict_data[0].keys()
df_by_source = pd.DataFrame.from_dict(dict_data, orient='index',columns=dict_keys)

df_by_source.head()

,date,traffic-source,traffic-details,visits,bounceRate,pageDepth,avgVisitSec,Goals_total
0,2022-06-20,Internal traffic,None,1.0,0.000000,1.000000,14.000000,0.0
1,2022-06-20,Direct traffic,None,8.0,12.500000,2.250000,183.250000,3.0
2,2022-06-20,Search engine traffic,Yandex,6.0,33.333333,1.666667,24.166667,1.0
3,2022-06-20,Ad traffic,Other ad: identified by tags,2.0,0.000000,7.000000,1005.500000,1.0
4,2022-06-20,Ad traffic,Yandex: Direct,17.0,17.647059,2.000000,212.470588,4.0


## **Запрос с группировкой по поисковой фразе**

In [60]:
# ЗАПРОС С ГРУППИРОВКОЙ ПО ПОИСКОВОЙ ФРАЗЕ
# МЕТРИКИ: ВИЗИТЫ, ПОЛЬЗОВАТЕЛИ, КОЛ-ВО ПРОСМ СТРАНИЦ, % НОВЫХ ПОЛЬЗ-Й, % ОТКАЗА, СР ГРУБИНА ПРОСМОТРА, СР ДЛИТ ВИЗИТА

# параметры запроса
params = dict(
    ids=count_number,  # номер счетчика в ЯМ
    metrics=current_metrics,
    # группировка по дате
    dimensions="ym:s:date, ym:s:<attribution>SearchPhrase",
    attribution="first",  # группировка по последнему значимому источнику
    date1='2022-06-20', 
    date2='2022-12-22',
    sort="ym:s:date",
    limit=10000,
    sccuracy = "full",      # максимальная точность отчетов
    proposed_accuracy = "false"     # система не может менять точность отчетов
)

#Получаем данные из Yandex.Metrika API
result = api.stats().get(params=params)
result = result().data

# из JSON файла оставляем только данные
result = result['data']

#Создаем пустой dict (словать данных)
dict_data = {}

#Парсим исходный list формата Json в dictionary (словарь данных)
for i in range(0, len(result)-1):
    dict_data[i] = {
            'date':result[i]["dimensions"][0]["name"],            # из списка dimensions, можно указывать не все параметры
            'searching_phrase':result[i]["dimensions"][1]["name"],
            #'traffic-details':result[i]["dimensions"][2]["name"],
            #'gender':result[i]["dimensions"][3]["name"],
            'visits':result[i]["metrics"][0],                     # из списа метрик
            'bounceRate':result[i]["metrics"][1],
            'pageDepth':result[i]["metrics"][2],
            'avgVisitSec':result[i]["metrics"][3],
            'Goals_total':result[i]["metrics"][4]
           # 'avgVisitDurationSeconds':result[i]["metrics"][5]

          }

#Создаем DataFrame из dict (словаря данных или массива данных)
dict_keys = dict_data[0].keys()
df_date_phrase = pd.DataFrame.from_dict(dict_data, orient='index',columns=dict_keys)

df_date_phrase.head()

,date,searching_phrase,visits,bounceRate,pageDepth,avgVisitSec,Goals_total
0,2022-06-20,None,29.0,13.793103,2.413793,246.724138,9.0
1,2022-06-20,касса лайф,1.0,0.000000,2.000000,21.000000,0.0
2,2022-06-20,туркасса,4.0,50.000000,1.250000,18.000000,0.0
3,2022-06-21,None,26.0,53.846154,1.230769,33.653846,1.0
4,2022-06-21,облачная касса от артура морозова,1.0,100.000000,1.000000,0.000000,0.0


## **Запрос с группировкой по данным из Директа**

In [61]:
# ЗАПРОС С ГРУППИРОВКОЙ ПО ДАТЕ,КАМПАНИИ ДИРЕКТА, ОБЪЯВЛЕНИЯ ДИРЕКТА, ТИП ПЛАТФОРМЫ И ПЛАТФОРМА ДИРЕКТА
# МЕТРИКИ: ВИЗИТЫ, ПОЛЬЗОВАТЕЛИ, КОЛ-ВО ПРОСМ СТРАНИЦ, % НОВЫХ ПОЛЬЗ-Й, % ОТКАЗА, СР ГРУБИНА ПРОСМОТРА, СР ДЛИТ ВИЗИТА

# параметры запроса
params = dict(
    ids=count_number,  # номер счетчика в ЯМ
    metrics=current_metrics,
    # группировка по дате
    dimensions="ym:s:date, ym:s:<attribution>DirectClickOrder, ym:s:<attribution>DirectClickBanner, ym:s:<attribution>DirectPlatformType, ym:s:<attribution>DirectPlatform",
    attribution="lastSign",  # группировка по последнему значимому источнику
    date1='2022-06-20',
    date2='2022-12-22',
    sort="ym:s:date",
    limit=10000,
    sccuracy = "full",      # максимальная точность отчетов
    proposed_accuracy = "false"     # система не может менять точность отчетов
)

# Получаем данные из Yandex.Metrika API
result = api.stats().get(params=params)
result = result().data

# из JSON файла оставляем только данные
result = result['data']

# Создаем пустой dict (словать данных)
dict_data = {}

# Парсим исходный list формата Json в dictionary (словарь данных)
for i in range(0, len(result)-1):
    dict_data[i] = {
        # из списка dimensions, можно указывать не все параметры
        'date': result[i]["dimensions"][0]["name"],
        'direct_company': result[i]["dimensions"][1]["name"],
        #'direct_adv_group': result[i]["dimensions"][2]["name"],
        'direct_adv': result[i]["dimensions"][2]["name"],
        'platform_type': result[i]["dimensions"][3]["name"],
        'platform': result[i]["dimensions"][4]["name"],
        # из списа метрик
        'visits':result[i]["metrics"][0],                     # из списа метрик
        'bounceRate':result[i]["metrics"][1],
         'pageDepth':result[i]["metrics"][2],
         'avgVisitSec':result[i]["metrics"][3],
         'Goals_total':result[i]["metrics"][4]
           # 'avgVisitDurationSeconds':result[i]["metrics"][5]

    }

# Создаем DataFrame из dict (словаря данных или массива данных)
dict_keys = dict_data[0].keys()
df_company_etc = pd.DataFrame.from_dict(
    dict_data, orient='index', columns=dict_keys)

df_company_etc.head()

,date,direct_company,direct_adv,platform_type,platform,visits,bounceRate,pageDepth,avgVisitSec,Goals_total
0,2022-06-20,None,None,None,None,20.0,20.0,2.45,208.55,5.0
1,2022-06-20,Kassa.life (Рся),Фискальный накопитель (ФН) в СПб,Networks,mail.yandex.ru,1.0,0.0,1.00,15.00,0.0
2,2022-06-20,Кассы / РСЯ,Онлайн-касса Эвотор 5 за 8900 руб! Б/У. Гарантия 3 мес.,Networks,com.DVloper.Granny3,1.0,0.0,2.00,699.00,0.0
3,2022-06-20,Кассы / РСЯ,"Онлайн-касса Эвотор 5 за 8900 руб! Состояние ""как новая""",Networks,m.news.yandex.ru,1.0,0.0,1.00,15.00,0.0
4,2022-06-20,Фискальный накопитель v2,Замена фискального накопителя СПб.,Search,yandex.ru,2.0,0.0,2.00,293.00,2.0


## **Запрос с группировкой по страницам входа и выхода**

In [62]:
# ЗАПРОС С ГРУППИРОВКОЙ ПО СТРАНИЦАМ ВХОДА И ВЫХОДА
# МЕТРИКИ: ВИЗИТЫ, ПОЛЬЗОВАТЕЛИ, КОЛ-ВО ПРОСМ СТРАНИЦ, % НОВЫХ ПОЛЬЗ-Й, % ОТКАЗА, СР ГРУБИНА ПРОСМОТРА, СР ДЛИТ ВИЗИТА

# параметры запроса
params = dict(
    ids=count_number,  # номер счетчика в ЯМ
    metrics=current_metrics,
    # группировка по дате
    dimensions="ym:s:date, ym:s:startURL, ym:s:endURL",
    attribution="lastSign",  # группировка по последнему значимому источнику
    date1='2022-06-20',
    date2='2022-12-22',
    sort="ym:s:date",
    limit=10000,
    sccuracy = "full",      # максимальная точность отчетов
    proposed_accuracy = "false"     # система не может менять точность отчетов
)

# Получаем данные из Yandex.Metrika API
result = api.stats().get(params=params)
result = result().data

# из JSON файла оставляем только данные
result = result['data']

# Создаем пустой dict (словать данных)
dict_data = {}

# Парсим исходный list формата Json в dictionary (словарь данных)
for i in range(0, len(result)-1):
    dict_data[i] = {
        # из списка dimensions, можно указывать не все параметры
        'date': result[i]["dimensions"][0]["name"],
        'first_page': result[i]["dimensions"][1]["name"],
        'last_page': result[i]["dimensions"][2]["name"],
        #'direct_adv': result[i]["dimensions"][2]["name"],
        #'platform_type': result[i]["dimensions"][3]["name"],
        #'platform': result[i]["dimensions"][4]["name"],
        # из списа метрик
         'visits':result[i]["metrics"][0],                     # из списа метрик
         'bounceRate':result[i]["metrics"][1],
         'pageDepth':result[i]["metrics"][2],
         'avgVisitSec':result[i]["metrics"][3],
         'Goals_total':result[i]["metrics"][4]
           # 'avgVisitDurationSeconds':result[i]["metrics"][5]

    }

# Создаем DataFrame из dict (словаря данных или массива данных)
dict_keys = dict_data[0].keys()
df_pages = pd.DataFrame.from_dict(dict_data, orient='index', columns=dict_keys)

df_pages.head()

,date,first_page,last_page,visits,bounceRate,pageDepth,avgVisitSec,Goals_total
0,2022-06-20,https://kassa.life/,https://kassa.life/,1.0,100.0,1.0,0.0,0.0
1,2022-06-20,https://kassa.life/,https://kassa.life/spec/tourism/turkassa,1.0,0.0,2.0,19.0,0.0
2,2022-06-20,https://kassa.life/,https://kassa.life/tilda/form172227518/payment/,1.0,0.0,6.0,116.0,3.0
3,2022-06-20,https://kassa.life/,https://kassa.life/vibrat-kassu,2.0,0.0,2.5,268.5,0.0
4,2022-06-20,https://kassa.life/,https://kassa.life/zamena-fn,2.0,0.0,3.0,126.0,2.0


## **Запрос с группировкой по рекламным системам**

In [63]:
# ЗАПРОС С ГРУППИРОВКОЙ ПО РЕКЛАМНЫМ СИСТЕМАМ
# МЕТРИКИ: ВИЗИТЫ, ПОЛЬЗОВАТЕЛИ, КОЛ-ВО ПРОСМ СТРАНИЦ, % НОВЫХ ПОЛЬЗ-Й, % ОТКАЗА, СР ГРУБИНА ПРОСМОТРА, СР ДЛИТ ВИЗИТА

# параметры запроса
params = dict(
    ids=count_number,  # номер счетчика в ЯМ
    metrics=current_metrics,
    # группировка по дате
    dimensions="ym:s:date, ym:s:<attribution>AdvEngine",
    attribution="lastSign",  # группировка по последнему значимому источнику
    date1='2022-06-20',
    date2='2022-12-22',
    sort="ym:s:date",
    limit=10000,
    sccuracy = "full",      # максимальная точность отчетов
    proposed_accuracy = "false"     # система не может менять точность отчетов
)

# Получаем данные из Yandex.Metrika API
result = api.stats().get(params=params)
result = result().data

# из JSON файла оставляем только данные
result = result['data']

# Создаем пустой dict (словать данных)
dict_data = {}

# Парсим исходный list формата Json в dictionary (словарь данных)
for i in range(0, len(result)-1):
    dict_data[i] = {
        # из списка dimensions, можно указывать не все параметры
        'date': result[i]["dimensions"][0]["name"],
        'Рекл_система': result[i]["dimensions"][1]["name"],
        #'last_page': result[i]["dimensions"][2]["name"],
        #'direct_adv': result[i]["dimensions"][2]["name"],
        #'platform_type': result[i]["dimensions"][3]["name"],
        #'platform': result[i]["dimensions"][4]["name"],
        # из списа метрик
        'visits':result[i]["metrics"][0],                     # из списа метрик
         'bounceRate':result[i]["metrics"][1],
         'pageDepth':result[i]["metrics"][2],
         'avgVisitSec':result[i]["metrics"][3],
         'Goals_total':result[i]["metrics"][4]
           # 'avgVisitDurationSeconds':result[i]["metrics"][5]
    }

# Создаем DataFrame из dict (словаря данных или массива данных)
dict_keys = dict_data[0].keys()
df_system = pd.DataFrame.from_dict(dict_data, orient='index', columns=dict_keys)

df_system.head(5)

,date,Рекл_система,visits,bounceRate,pageDepth,avgVisitSec,Goals_total
0,2022-06-20,None,15.0,20.000000,1.933333,108.333333,4.0
1,2022-06-20,Other ad: identified by tags,2.0,0.000000,7.000000,1005.500000,1.0
2,2022-06-20,Yandex: Direct,17.0,17.647059,2.000000,212.470588,4.0
3,2022-06-21,None,3.0,33.333333,1.333333,52.333333,0.0
4,2022-06-21,Yandex: Direct,25.0,56.000000,1.200000,29.320000,1.0


## **Запрос с данными по конверсии по кампаниям**

In [64]:
# ЗАПРОС С ГРУППИРОВКОЙ ПО КАМПАНИИ ДИРЕКТА, ОБЪЯВЛЕНИЯ ДИРЕКТА, ТИП ПЛАТФОРМЫ И ПЛАТФОРМА ДИРЕКТА
# МЕТРИКИ: ВИЗИТЫ, ПОЛЬЗОВАТЕЛИ, % ОТКАЗА, СР ГРУБИНА ПРОСМОТРА, СР ДЛИТ ВИЗИТА, ДОЛЯ ЦЕЛЕВЫХ ВИЗИТОВ, КОЛ-ВО ЦЕЛЕВЫХ ВИЗИТОВ
# ЦЕЛЬ (202896550) - КЛИК ПО НОМЕРУ ТЕЛЕФОНА

# параметры запроса
params = dict(
    ids=count_number,  # номер счетчика в ЯМ
    metrics=current_metrics,
    # группировка по дате
    dimensions="ym:s:<attribution>DirectClickOrder",
    attribution="lastSign",  # группировка по последнему значимому источнику
    #goal_id = 202896550,
    date1='2022-06-20',
    date2='2022-12-22',
    #sort="ym:s:date",
    limit=10000,
    sccuracy = "full",      # максимальная точность отчетов
    proposed_accuracy = "false"     # система не может менять точность отчетов
)

# Получаем данные из Yandex.Metrika API
result = api.stats().get(params=params)
result = result().data

# из JSON файла оставляем только данные
result = result['data']

# Создаем пустой dict (словать данных)
dict_data = {}

# Парсим исходный list формата Json в dictionary (словарь данных)
for i in range(0, len(result)-1):
    dict_data[i] = {
        # из списка dimensions, можно указывать не все параметры
        # 'date': result[i]["dimensions"][0]["name"],
        'direct_company': result[i]["dimensions"][0]["name"],
        # 'direct_adv_group': result[i]["dimensions"][2]["name"],
        #'direct_adv': result[i]["dimensions"][1]["name"],
        #'platform_type': result[i]["dimensions"][2]["name"],
        #'platform': result[i]["dimensions"][3]["name"],
        # из списа метрик
        'visits':result[i]["metrics"][0],                     # из списа метрик
         'bounceRate':result[i]["metrics"][1],
         'pageDepth':result[i]["metrics"][2],
         'avgVisitSec':result[i]["metrics"][3],
         'Goals_total':result[i]["metrics"][4]
        # 'avgVisitDurationSeconds':result[i]["metrics"][5]
    }

# Создаем DataFrame из dict (словаря данных или массива данных)
dict_keys = dict_data[0].keys()
df_goals_company = pd.DataFrame.from_dict(
    dict_data, orient='index', columns=dict_keys)

df_goals_company.head()

,direct_company,visits,bounceRate,pageDepth,avgVisitSec,Goals_total
0,ФН / Поиск / СПб / ОК,484.0,14.876033,1.572314,100.549587,136.0
1,Копия ФН / Поиск / СПб (05/07),452.0,18.584071,1.373894,87.590708,123.0
2,Аренда МК / Ш / 700р — 2,318.0,52.201258,1.078616,44.789308,6.0
3,ФН / МК / Владимир / Широкие - 1800р — 2,317.0,62.460568,1.154574,27.164038,65.0
4,ФН / МК / Владимир / Целевые / CPC: 3к/нед - 14.12,227.0,65.638767,1.092511,36.458150,9.0


In [65]:
# округлим столбцы до 2 знаков после запятой
for col in ['bounceRate', 'pageDepth', 'avgVisitSec']:
    df_goals_company[col] = round(df_goals_company[col] , 2)

## **Запрос с данными по конверсии по объявлениям**

In [66]:
# ЗАПРОС С ГРУППИРОВКОЙ ПО КАМПАНИИ ДИРЕКТА, ОБЪЯВЛЕНИЯ ДИРЕКТА, ТИП ПЛАТФОРМЫ И ПЛАТФОРМА ДИРЕКТА
# МЕТРИКИ: ВИЗИТЫ, ПОЛЬЗОВАТЕЛИ, % ОТКАЗА, СР ГРУБИНА ПРОСМОТРА, СР ДЛИТ ВИЗИТА, ДОЛЯ ЦЕЛЕВЫХ ВИЗИТОВ, КОЛ-ВО ЦЕЛЕВЫХ ВИЗИТОВ
# ЦЕЛЬ (202896550) - КЛИК ПО НОМЕРУ ТЕЛЕФОНА

# параметры запроса
params = dict(
    ids=count_number,  # номер счетчика в ЯМ
    metrics=current_metrics,
    # группировка по дате
    dimensions="ym:s:<attribution>DirectClickBanner",
    attribution="lastSign",  # группировка по последнему значимому источнику
    #goal_id = 202896550,
    date1='2022-06-20',
    date2='2022-12-22',
    #sort="ym:s:date",
    limit=10000,
    sccuracy = "full",      # максимальная точность отчетов
    proposed_accuracy = "false"     # система не может менять точность отчетов
)

# Получаем данные из Yandex.Metrika API
result = api.stats().get(params=params)
result = result().data

# из JSON файла оставляем только данные
result = result['data']

# Создаем пустой dict (словать данных)
dict_data = {}

# Парсим исходный list формата Json в dictionary (словарь данных)
for i in range(0, len(result)-1):
    dict_data[i] = {
        # из списка dimensions, можно указывать не все параметры
        # 'date': result[i]["dimensions"][0]["name"],
        'direct_adv': result[i]["dimensions"][0]["name"],
        # 'direct_adv_group': result[i]["dimensions"][2]["name"],
        #'direct_adv': result[i]["dimensions"][1]["name"],
        #'platform_type': result[i]["dimensions"][2]["name"],
        #'platform': result[i]["dimensions"][3]["name"],
        # из списа метрик
        'visits':result[i]["metrics"][0],                     # из списа метрик
         'bounceRate':result[i]["metrics"][1],
         'pageDepth':result[i]["metrics"][2],
         'avgVisitSec':result[i]["metrics"][3],
         'Goals_total':result[i]["metrics"][4]
        # 'avgVisitDurationSeconds':result[i]["metrics"][5]
    }

# Создаем DataFrame из dict (словаря данных или массива данных)
dict_keys = dict_data[0].keys()
df_goals_adv = pd.DataFrame.from_dict(
    dict_data, orient='index', columns=dict_keys)

df_goals_adv.head()

,direct_adv,visits,bounceRate,pageDepth,avgVisitSec,Goals_total
0,Фискальный накопитель на 16 и 36 мес. Всего от 9400 руб!,120.0,67.500000,1.116667,23.641667,13.0
1,Купить фискальный накопитель 15/36. Всего от 9400 руб!,115.0,71.304348,1.043478,23.026087,4.0
2,Купить фискальный накопитель 15/36.,113.0,14.159292,1.530973,122.008850,35.0
3,Касса в аренду за 1400 ₽/мес. Начните работать сегодня!,105.0,54.285714,1.057143,30.190476,3.0
4,Купить фискальный накопитель 36 мес.,90.0,13.333333,1.711111,123.955556,20.0


In [67]:
# округлим столбцы до 2 знаков после запятой
for col in ['bounceRate', 'pageDepth', 'avgVisitSec']:
    df_goals_adv[col] = round(df_goals_adv[col] , 2)

## *Функция группировки данных по одному параметру (group_df)*

In [100]:
def group_df(df, column):
    # сгруппируем количество показов, кликов, СТР и стоимости по устройствам
    grouped_device = df.groupby(column).agg({'visits': ['sum'], 'bounceRate': ['mean'], 
                                             'pageDepth': ['mean'], 'avgVisitSec': ['mean'], 'Goals_total': ['sum']}).reset_index()
    grouped_device.columns = [column, 'Визитов', '% откзов', 'Ср глубина просмотра', 'Ср сессия в сек', 'Достигнуто целей']
    
    grouped_device['Конверсия'] = (grouped_device['Достигнуто целей'] / grouped_device['Визитов']) * 100
    for col in ('% откзов', 'Ср глубина просмотра', 'Ср сессия в сек', 'Конверсия'):
        grouped_device[col] = round((grouped_device[col]), 2)
    
    return(grouped_device)

## *Функция группировки по двум пар-м (group_df_2_param)*

In [104]:
def group_df_2_param(df, column1, column2):
    # сгруппируем количество показов, кликов, СТР и стоимости по устройствам
    grouped_device = df.groupby([column1, column2] ).agg({'visits': ['sum'], 'bounceRate': ['mean'], 
                                             'pageDepth': ['mean'], 'avgVisitSec': ['mean'], 'Goals_total': ['sum']}).reset_index()
    grouped_device.columns = [column1, column2, 'Визитов', '% откзов', 'Ср глубина просмотра', 'Ср сессия в сек', 'Достигнуто целей']

    grouped_device['Конверсия'] = (grouped_device['Достигнуто целей'] / grouped_device['Визитов']) * 100
    for col in ('% откзов', 'Ср глубина просмотра', 'Ср сессия в сек', 'Конверсия'):
        grouped_device[col] = round((grouped_device[col]), 2)
        
    return(grouped_device)